In [1]:
import pandas as pd
from math import sqrt
from scipy.stats import pearsonr
from sklearn.preprocessing import scale
from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate, KFold

## Part I: Training a classifier based on the PMEmo dataset
### Loading the training data
The raw dataset consists of 794 songs × 6374 features, including 767 labeled songs.  
Select 260 core features  
Core features: the average and the standard deviation of the 65 LLDs as well as their first-order derivatives

In [2]:
def select_core_features(data):
    col = [0] * 260
    for i in range(118):
        col[i*2] = 16 + i*31
        col[i*2+1] = 19 + i*31
        
    for j in range(12):
        col[j*2 + 236] = 3659 + j*39
        col[j*2 + 237] = 3683 + j*39
        
    return scale(data.iloc[:, col].values)

In [3]:
feature = pd.read_csv("./training data/static_features.csv")
annotation = pd.read_csv("./training data/static_annotations.csv")
data = pd.merge(feature, annotation)
X_training, y_a, y_v = select_core_features(data), data.loc[:, 'Arousal(mean)'].values, data.loc[:, 'Valence(mean)'].values

### Setting up Metric & Regression Classifiers
Metric: (1) Root Mean Square Error(RMSE); (2) Pearson Correlation Coefficient(PCC)  
Regressor: Lasso, Ridge, Elastic Net, SVR(kernel = rbf/linear/poly)

In [4]:
def rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))

def pcc(y, y_pred):
    return pearsonr(y, y_pred)[0]

Regressors = {
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet(),
    'SVRrbf': SVR(kernel='rbf', gamma='scale'),
    #'SVRlinear': SVR(kernel='linear', gamma='scale'),
    #'SVRpoly': SVR(kernel='poly', gamma='scale'),
    'DT': DecisionTreeRegressor(max_depth=5),
    'RF': RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1)
}

### Training Classifiers & Evaluation

In [5]:
def cross_val_regression(regressors, features, labels):
    res = pd.DataFrame(columns = list(Regressors.keys()))
    scorer = {'rmse': make_scorer(rmse)}
    
    for name, reg in Regressors.items():
        reg_score = cross_validate(reg, 
                                   features, 
                                   labels, 
                                   scoring = scorer,
                                   cv = 10, 
                                   return_train_score = False,
                                   error_score = 'raise') 
        res.loc['RMSE', name] = round(reg_score['test_rmse'].mean(), 3)
        #res.loc['PCC', name] = reg_score['test_pcc'].mean()
    return res

In [6]:
scores_Arousal = cross_val_regression(Regressors, X_training, y_a)
scores_Arousal

,Lasso,Ridge,ElasticNet,SVRrbf,DT,RF
RMSE,0.184,0.185,0.184,0.134,0.163,0.169


In [7]:
scores_Valence = cross_val_regression(Regressors, X_training, y_v)
scores_Valence

,Lasso,Ridge,ElasticNet,SVRrbf,DT,RF
RMSE,0.162,0.183,0.162,0.135,0.16,0.149


## Part II: Recognize emotions from new music segments
### Loading the testing data

In [8]:
data = pd.read_csv("./testing data/static_features.csv")
X_testing = select_core_features(data)

### Predicting arousal and valence of new input

In [9]:
res_a = pd.DataFrame()
res_v = pd.DataFrame()
res_a['musicId'] = res_v['musicId'] = data.iloc[:, 0]

#Predict arousal
for name, reg in Regressors.items():
    reg.fit(X_training, y_a)
    res_a[name] = reg.predict(X_testing)
    
#Predict Valence
for name, reg in Regressors.items():
    reg.fit(X_training, y_v)
    res_v[name] = reg.predict(X_testing)

In [10]:
res_a

,musicId,Lasso,Ridge,ElasticNet,SVRrbf,DT,RF
0,'2',0.622355,0.742548,0.622355,0.676671,0.774545,0.562888
1,'3',0.622355,0.502163,0.622355,0.548987,0.787500,0.571326


In [11]:
res_v

,musicId,Lasso,Ridge,ElasticNet,SVRrbf,DT,RF
0,'2',0.596581,0.712793,0.596581,0.669296,0.673437,0.577771
1,'3',0.596581,0.480368,0.596581,0.492799,0.519744,0.559609
